# Data cleansing

In [1]:
import pandas as pd
import pickle
from tqdm import tqdm
import numpy as np
import warnings

# Tắt hiển thị FutureWarning
warnings.simplefilter(action='ignore', category=FutureWarning)

# 1.Data chotot

In [3]:
# Import data from pickle
with open('car_detail_chotot.pkl','rb') as f:
    chotot = pickle.load(f)

In [4]:
chotot["web"] = "chotot"

In [5]:
# Summarize data
print(chotot.info())
chotot.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 19920 entries, 0 to 0
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           19920 non-null  object
 1   post_date     19920 non-null  object
 2   Price         19920 non-null  int64 
 3   carbrand      19920 non-null  object
 4   carmodel      19920 non-null  object
 5   mfdate        19920 non-null  object
 6   mileage_v2    19920 non-null  object
 7   condition_ad  19920 non-null  object
 8   gearbox       19920 non-null  object
 9   fuel          19920 non-null  object
 10  carorigin     19920 non-null  object
 11  cartype       19920 non-null  object
 12  carseats      19920 non-null  object
 13  web           19920 non-null  object
dtypes: int64(1), object(13)
memory usage: 2.3+ MB
None


,url,post_date,Price,carbrand,carmodel,mfdate,mileage_v2,condition_ad,gearbox,fuel,carorigin,cartype,carseats,web
0,https://xe.chotot.com/mua-ban-oto-quan-tan-bin...,2 tháng trước,500000000,Toyota,Vios,2022,9996,Đã sử dụng,Tự động,Xăng,Việt Nam,,,chotot
0,https://xe.chotot.com/mua-ban-oto-quan-tan-phu...,23 giờ trước,979000000,Ford,Ranger,2024,,Mới,Tự động,Xăng,Việt Nam,Pick-up (bán tải),,chotot
0,https://xe.chotot.com/mua-ban-oto-quan-tan-phu...,11 giờ trước,950000000,Ford,Everest,2019,38000,Đã sử dụng,Tự động,Dầu,Thái Lan,,,chotot


In [6]:
# Kiểm tra các giá trị unique
chotot.nunique().sort_values(ascending=False)

url             19905
mileage_v2       1931
Price            1668
carmodel          450
post_date          95
carbrand           68
mfdate             44
carorigin          12
carseats           11
cartype            10
gearbox             5
fuel                4
condition_ad        2
web                 1
dtype: int64

## 1.1.Post_date

In [7]:
# Kiểm tra trường post_date
chotot["post_date"].value_counts()

post_date
2 tuần trước     3064
1 tháng trước    2543
3 tuần trước     2432
1 tuần trước     1652
4 tuần trước     1602
                 ... 
59 phút trước       1
14 phút trước       1
27 phút trước       1
7 phút trước        1
5 giờ trước         1
Name: count, Length: 95, dtype: int64

In [8]:
# Chuyển giá trị "hôm qua" thành "1 ngày trước", sau đó xóa phần " trước"
chotot["post_date"] = chotot["post_date"].replace({"hôm qua":"1 ngày trước"}).str[:-6]

In [9]:
# Tách thông tin thời gian và unit
chotot[["time", "unit"]] = chotot["post_date"].str.split(" ", expand=True)
chotot["unit"].value_counts()

unit
tuần     8750
ngày     5870
tháng    2544
giờ      2489
phút      267
Name: count, dtype: int64

In [10]:
# Chuyển thông tin thành dạng ngày tháng
def update_post_date(row):
    if row["unit"] in ["giờ", "phút"]:
        return pd.to_datetime("2024-04-02", format="%Y-%m-%d")
    elif row["unit"] == "ngày":
        return pd.to_datetime("2024-04-02", format="%Y-%m-%d") - pd.Timedelta(days=int(row["time"]))
    elif row["unit"] == "tuần":
        return pd.to_datetime("2024-04-02", format='%Y-%m-%d') - pd.Timedelta(weeks=int(row["time"]))
    else:
        return pd.to_datetime("2024-04-02", format="%Y-%m-%d") - pd.Timedelta(days=30*int(row["time"]))

chotot["post_date"] = chotot.apply(update_post_date, axis=1)
chotot.drop(['time', 'unit'] , inplace = True ,axis=1)

## 1.2.Năm sản xuất

In [11]:
chotot["mfdate"].value_counts()

mfdate
2022              2382
2023              1853
2021              1826
2019              1678
2020              1538
2024              1399
2018              1350
2016              1049
2017               983
2015               833
2014               578
2009               563
2010               481
2008               444
2011               436
2007               362
2013               307
2004               302
2012               280
2003               264
2005               243
2006               152
2002               139
2001                87
2000                82
1993                34
1997                32
trước năm 1980      31
1995                29
1992                28
1996                24
1999                23
1998                22
1991                16
1994                14
1990                12
1987                10
1989                 8
1984                 8
1988                 6
1985                 6
1986                 4
1982                 1
1983

In [12]:
# Tồn tại giá trị "trước năm 1980" -> Chuyển về 1980 sau đó chuyển dạng int
chotot.loc[:,"mfdate"] = chotot["mfdate"].replace({"trước năm 1980":"1980"}).astype(int)
chotot["mfdate"].info()

<class 'pandas.core.series.Series'>
Index: 19920 entries, 0 to 0
Series name: mfdate
Non-Null Count  Dtype 
--------------  ----- 
19920 non-null  object
dtypes: object(1)
memory usage: 311.2+ KB


## 1.3.Số KM đã đi

In [13]:
chotot["mileage_v2"].value_counts()

mileage_v2
          2738
100000     557
50000      431
80000      398
60000      378
          ... 
83120        1
60500        1
229000       1
68420        1
84376        1
Name: count, Length: 1931, dtype: int64

In [14]:
# Chuyển định dạng trường mileage_v2 về int
chotot.loc[:,"mileage_v2"] = chotot["mileage_v2"].replace({"":np.nan}).astype(float)
chotot["mileage_v2"].info()

<class 'pandas.core.series.Series'>
Index: 19920 entries, 0 to 0
Series name: mileage_v2
Non-Null Count  Dtype 
--------------  ----- 
17182 non-null  object
dtypes: object(1)
memory usage: 311.2+ KB


## 1.4.Tình trạng

In [15]:
chotot["condition_ad"].value_counts()

condition_ad
Đã sử dụng    17484
Mới            2436
Name: count, dtype: int64

## 1.5.Hộp số

In [16]:
chotot["gearbox"].value_counts()

gearbox
Tự động        14706
Số sàn          5041
5                102
Bán tự động       63
4                  8
Name: count, dtype: int64

In [17]:
# Kiểm tra các trường hợp có gearbox là "4", "5"
check_gearbox = chotot.loc[chotot["gearbox"].isin(["4","5"]),["carmodel","carbrand"]].value_counts().reset_index()
full_gearbox = chotot.loc[:,["carmodel","carbrand","gearbox"]].value_counts().reset_index().rename(columns={"count":"count_full"})

check_gearbox.merge(full_gearbox)

,carmodel,carbrand,count,gearbox,count_full
0,Vios,Toyota,32,Số sàn,304
1,Vios,Toyota,32,Tự động,283
2,Vios,Toyota,32,5,32
3,Vios,Toyota,32,Bán tự động,3
4,Corolla Cross,Toyota,27,Tự động,159
5,Corolla Cross,Toyota,27,5,27
6,Corolla Cross,Toyota,27,Số sàn,2
7,Corolla Cross,Toyota,27,Bán tự động,1
8,CR V,Honda,14,Tự động,219
9,CR V,Honda,14,5,14


In [18]:
# Kiểm tra các xe có thể có số sàn thì đều số tự động -> gán số tự động
chotot.loc[:,"gearbox"] = chotot["gearbox"].replace(["4","5"],"Tự động")
chotot.gearbox.value_counts()

gearbox
Tự động        14816
Số sàn          5041
Bán tự động       63
Name: count, dtype: int64

## 1.6.Nhiên liệu

In [19]:
chotot["fuel"].value_counts()

fuel
Xăng              16271
Dầu                3278
Điện                207
Động cơ Hybrid      164
Name: count, dtype: int64

## 1.7.Nơi sản xuất

In [20]:
# carorigin
chotot["carorigin"].value_counts()

carorigin
Việt Nam         9135
Thái Lan         2559
                 1886
Nước khác        1625
Hàn Quốc         1182
Nhật Bản         1146
Đang cập nhật     927
Đức               605
Mỹ                501
Ấn Độ             196
Trung Quốc        109
Đài Loan           49
Name: count, dtype: int64

In [21]:
# Chuyển các giá trị null, "Đang cập nhật" về dạng NaN
chotot.loc[:,"carorigin"] = chotot["carorigin"].replace(["Đang cập nhật",""],np.nan)
chotot.carorigin.value_counts()

carorigin
Việt Nam      9135
Thái Lan      2559
Nước khác     1625
Hàn Quốc      1182
Nhật Bản      1146
Đức            605
Mỹ             501
Ấn Độ          196
Trung Quốc     109
Đài Loan        49
Name: count, dtype: int64

## 1.8.Kiểu dáng

In [22]:
# cartype
chotot["cartype"].value_counts()

cartype
                     15822
SUV / Cross over      1597
Sedan                 1235
Minivan (MPV)          620
Hatchback              308
Pick-up (bán tải)      222
Van                     52
Kiểu dáng khác          48
Coupe (2 cửa)           13
Mui trần                 3
Name: count, dtype: int64

In [23]:
# Chuyển về dạng NaN đối với cartype trống, "Kiểu dáng khác"
chotot.loc[:,"cartype"] = chotot["cartype"].replace(["","Kiểu dáng khác"],np.nan)
chotot["cartype"].value_counts()

cartype
SUV / Cross over     1597
Sedan                1235
Minivan (MPV)         620
Hatchback             308
Pick-up (bán tải)     222
Van                    52
Coupe (2 cửa)          13
Mui trần                3
Name: count, dtype: int64

## 1.9.Số chỗ ngồi

In [24]:
# carseats
chotot["carseats"].value_counts()


carseats
        19440
5         267
7         134
16         27
8          16
4           9
6           9
2           9
Khác        5
9           3
12          1
Name: count, dtype: int64

In [25]:
# Chuyển các trường hợp trống về dạng NA và chuyển sang float
chotot.loc[:,"carseats"] = chotot["carseats"].replace(["","Khác"],np.nan).astype(float)
chotot["carseats"].value_counts()

carseats
5.0     267
7.0     134
16.0     27
8.0      16
4.0       9
6.0       9
2.0       9
9.0       3
12.0      1
Name: count, dtype: int64

# 2.Data bonbanh

In [26]:
# Import data from pickle
with open('car_detail_bonbanh.pkl','rb') as f:
    bonbanh = pickle.load(f)

In [27]:
# Đổi tên trường cho giống chotot
bonbanh = bonbanh.rename(columns={'price':'Price', 'brand':'carbrand', 'model':'carmodel', 'Năm sản xuất':'mfdate', 'Số Km đã đi':'mileage_v2',
                                  'Tình trạng':'condition_ad', 'Hộp số':'gearbox', 'Động cơ':'fuel', 'Xuất xứ':'carorigin', 'Kiểu dáng':'cartype',
                                  'Số chỗ ngồi':'carseats', 'Màu ngoại thất':'col_out', 'Màu nội thất':'col_in','Số cửa':'carwindows',
                                  'Dẫn động':'drive'})

In [28]:
bonbanh["web"] = "bonbanh"

In [29]:
# Summarize data
print(bonbanh.info())
bonbanh.head(3)

<class 'pandas.core.frame.DataFrame'>
Index: 29757 entries, 0 to 0
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   url           29757 non-null  object
 1   post_date     29757 non-null  object
 2   Price         29757 non-null  object
 3   carbrand      29757 non-null  object
 4   carmodel      29757 non-null  object
 5   mfdate        29757 non-null  object
 6   condition_ad  29757 non-null  object
 7   mileage_v2    22979 non-null  object
 8   carorigin     29757 non-null  object
 9   cartype       29757 non-null  object
 10  gearbox       29757 non-null  object
 11  fuel          29757 non-null  object
 12  col_out       29757 non-null  object
 13  col_in        29757 non-null  object
 14  carseats      29757 non-null  object
 15  carwindows    29757 non-null  object
 16  drive         29757 non-null  object
 17  web           29757 non-null  object
dtypes: object(18)
memory usage: 4.3+ MB
None


,url,post_date,Price,carbrand,carmodel,mfdate,condition_ad,mileage_v2,carorigin,cartype,gearbox,fuel,col_out,col_in,carseats,carwindows,drive,web
0,https://bonbanh.com/xe-toyota-yaris-1.3-at-200...,1/04/2024,220 Triệu,Toyota,Yaris,2007,Xe đã dùng,"120,000 Km",Nhập khẩu,Hatchback,Số tự động,Xăng 1.3 L,Ghi,Kem,5 chỗ,5 cửa,FWD - Dẫn động cầu trước,bonbanh
0,https://bonbanh.com/xe-toyota-land_cruiser-vx-...,1/04/2024,3 Tỷ 250 Triệu,Toyota,Land Cruiser,2016,Xe đã dùng,"50,000 Km",Nhập khẩu,SUV,Số tự động,Xăng 4.6 L,Trắng,Kem,8 chỗ,5 cửa,AWD - 4 bánh toàn thời gian,bonbanh
0,https://bonbanh.com/xe-mercedes_benz-glc-300-4...,1/04/2024,1 Tỷ 666 Triệu,Mercedes Benz,GLC,2020,Xe đã dùng,"55,000 Km",Lắp ráp trong nước,SUV,Số tự động,Xăng 2.0 L,Xanh,Đen,5 chỗ,5 cửa,AWD - 4 bánh toàn thời gian,bonbanh


In [30]:
# Kiểm tra các giá trị unique
bonbanh.nunique().sort_values(ascending=False)

url             29747
Price            2179
mileage_v2       1679
carmodel          480
post_date         117
fuel               97
carbrand           77
mfdate             36
carseats           19
col_in             18
col_out            18
cartype            10
carwindows          8
drive               5
gearbox             2
carorigin           2
condition_ad        2
web                 1
dtype: int64

## 2.1.Post_date

In [31]:
# Kiểm tra trường post_date
bonbanh["post_date"].value_counts()

post_date
2/04/2024     8740
1/04/2024     7037
31/03/2024    1753
30/03/2024     910
29/03/2024     611
              ... 
12/07/2019       1
29/08/2019       1
12/09/2019       1
10/12/2023       1
13/06/2019       1
Name: count, Length: 117, dtype: int64

In [32]:
# Chuyển về dạng date
bonbanh["post_date"] = pd.to_datetime(bonbanh["post_date"], format='%d/%m/%Y')

## 2.2.Năm sản xuất

In [33]:
bonbanh["mfdate"].value_counts()

mfdate
2024    4443
2022    3821
2021    3128
2023    2877
2019    2344
2020    2239
2018    1897
2016    1497
2017    1332
2015    1114
2014     799
2009     625
2010     591
2011     497
2013     479
2008     463
2007     344
2012     312
2004     197
2005     190
2006     150
2003     142
2002      66
2001      58
2000      39
1989      24
1992      14
1993      13
1995      12
1996       9
1999       9
1998       9
1997       8
1994       6
1991       5
1990       4
Name: count, dtype: int64

## 2.3.Số KM đã đi

In [34]:
bonbanh["mileage_v2"].value_counts()

mileage_v2
0 Km          3903
50,000 Km      718
60,000 Km      651
30,000 Km      640
40,000 Km      606
              ... 
15,139 Km        1
30,789 Km        1
118,170 Km       1
37,900 Km        1
7,129 Km         1
Name: count, Length: 1679, dtype: int64

In [35]:
bonbanh.loc[bonbanh["mileage_v2"]=="0 Km","mfdate"].value_counts()

mfdate
2019    318
2016    317
2018    272
2021    272
2015    271
2009    253
2017    220
2020    217
2022    215
2014    201
2008    176
2010    169
2011    169
2007    131
2012    125
2013    107
2005     77
2004     73
2003     60
2006     56
2023     54
2002     37
2001     34
1989     17
2000     17
1992      7
1993      7
1995      6
1999      5
1996      5
1998      4
1997      3
1994      3
2024      2
1990      2
1991      1
Name: count, dtype: int64

In [36]:
# Chuyển về định dạng float
bonbanh.loc[:,"mileage_v2"] = bonbanh['mileage_v2'].str.replace(' Km', '').str.replace(',', '').astype(float)

# Các trường hợp bằng 0 chuyển về nan (do các xe này đều sản xuất xa, không phải không đi km nào)
bonbanh.loc[:,"mileage_v2"] = bonbanh["mileage_v2"].replace({0:np.nan})

## 2.4.Tình trạng

In [37]:
bonbanh["condition_ad"].value_counts()

condition_ad
Xe đã dùng    22979
Xe mới         6778
Name: count, dtype: int64

In [38]:
# Cập nhật để thống nhất với bộ chotot
bonbanh.loc[:,"condition_ad"] = bonbanh["condition_ad"].replace({"Xe đã dùng":"Đã sử dụng", "Xe mới":"Mới"})
bonbanh["condition_ad"].value_counts()

condition_ad
Đã sử dụng    22979
Mới            6778
Name: count, dtype: int64

## 2.5.Hộp số

In [39]:
bonbanh["gearbox"].value_counts()

gearbox
Số tự động    25404
Số tay         4353
Name: count, dtype: int64

In [40]:
# Cập nhật để thống nhất với bộ chotot
bonbanh.loc[:,"gearbox"] = bonbanh["gearbox"].replace({"Số tự động":"Tự động", "Số tay":"Số sàn"})
bonbanh["gearbox"].value_counts()

gearbox
Tự động    25404
Số sàn      4353
Name: count, dtype: int64

## 2.6.Nhiên liệu

In [41]:
bonbanh["fuel"].value_counts()

fuel
Xăng  2.0 L      7003
Xăng  1.5 L      5234
Dầu  2.0 L       1579
Dầu  2.2 L       1412
Xăng  1.6 L      1376
                 ... 
Dầu  5.7 L          1
Hybrid  1.3 L       1
Dầu  11.1 L         1
Xăng  2.8 L         1
Dầu  3.4 L          1
Name: count, Length: 97, dtype: int64

In [42]:
# Lấy thông tin nhiên liệu
bonbanh.loc[:,"fuel"] = bonbanh["fuel"].apply(lambda x: x.split(" ")[0])
bonbanh["fuel"].value_counts()

fuel
Xăng      24136
Dầu        4751
Hybrid      456
Điện        413
-             1
Name: count, dtype: int64

In [43]:
# Cập nhật để thống nhất với bộ chotot
bonbanh.loc[:,"fuel"] = bonbanh["fuel"].replace({"Hybrid":"Động cơ Hybrid","-":np.nan})
bonbanh["fuel"].value_counts()

fuel
Xăng              24136
Dầu                4751
Động cơ Hybrid      456
Điện                413
Name: count, dtype: int64

## 2.7.Nơi sản xuất

In [44]:
# carorigin
bonbanh["carorigin"].value_counts()

carorigin
Lắp ráp trong nước    16727
Nhập khẩu             13030
Name: count, dtype: int64

## 2.8.Kiểu dáng

In [45]:
# cartype
bonbanh["cartype"].value_counts()

cartype
SUV                      11728
Sedan                     9178
Crossover                 2763
Hatchback                 2141
Bán tải / Pickup        1878
Van/Minivan               1485
Truck                      246
Coupe                      215
Convertible/Cabriolet      115
Wagon                        8
Name: count, dtype: int64

## 2.9.Số chỗ ngồi

In [46]:
# carseats
bonbanh["carseats"].value_counts()

carseats
5 chỗ     19879
7 chỗ      6933
8 chỗ      1118
4 chỗ       717
2 chỗ       381
16 chỗ      237
3 chỗ       199
6 chỗ       195
9 chỗ        38
10 chỗ       26
29 chỗ       16
12 chỗ        4
15 chỗ        4
0 chỗ         3
47 chỗ        2
40 chỗ        2
45 chỗ        1
43 chỗ        1
46 chỗ        1
Name: count, dtype: int64

In [47]:
# Chuyển về định dạng float
bonbanh.loc[:,"carseats"] = bonbanh["carseats"].str.replace(" chỗ", "").astype(float)

# Các trường hợp bằng 0 chuyển về nan
bonbanh.loc[:,"carseats"] = bonbanh["carseats"].replace({0:np.nan})
bonbanh["carseats"].value_counts()

carseats
5.0     19879
7.0      6933
8.0      1118
4.0       717
2.0       381
16.0      237
3.0       199
6.0       195
9.0        38
10.0       26
29.0       16
12.0        4
15.0        4
47.0        2
40.0        2
45.0        1
43.0        1
46.0        1
Name: count, dtype: int64

## 2.10.Price

In [48]:
# Price
bonbanh["Price"].value_counts()

Price
450 Triệu          176
465 Triệu          168
495 Triệu          157
485 Triệu          154
650 Triệu          149
                  ... 
3 Tỷ 91 Triệu        1
11 Tỷ 690 Triệu      1
2 Tỷ 349 Triệu       1
4 Tỷ 369 Triệu       1
2 Tỷ 686 Triệu       1
Name: count, Length: 2179, dtype: int64

In [49]:
# Chuyển sang dạng int
bonbanh["Price"] = bonbanh["Price"].apply(lambda x: 1000000*sum(list(map(int,x.replace(" Triệu","").replace(" Tỷ","000").split(" ")))))

## 2.11.Drop các trường không trùng bộ chotot

In [50]:
bonbanh.drop(["col_out","col_in","carwindows","drive"], axis=1, inplace=True)


# 3.Ghép 2 bộ dữ liệu

In [51]:
car = pd.concat([chotot,bonbanh], axis=0, ignore_index=True)
car.shape

(49677, 14)

In [52]:
# Kiểm tra thông tin các bảng dữ liệu
car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49677 entries, 0 to 49676
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   url           49677 non-null  object        
 1   post_date     49677 non-null  datetime64[ns]
 2   Price         49677 non-null  int64         
 3   carbrand      49677 non-null  object        
 4   carmodel      49677 non-null  object        
 5   mfdate        49677 non-null  object        
 6   mileage_v2    36258 non-null  object        
 7   condition_ad  49677 non-null  object        
 8   gearbox       49677 non-null  object        
 9   fuel          49676 non-null  object        
 10  carorigin     46864 non-null  object        
 11  cartype       33807 non-null  object        
 12  carseats      30229 non-null  object        
 13  web           49677 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(12)
memory usage: 5.3+ MB


In [53]:
# Kiểm tra nan trong bộ dữ liệu
car.isna().sum().sort_values(ascending=False)

carseats        19448
cartype         15870
mileage_v2      13419
carorigin        2813
fuel                1
url                 0
post_date           0
Price               0
carbrand            0
carmodel            0
mfdate              0
condition_ad        0
gearbox             0
web                 0
dtype: int64

Kiểm tra giá trị trong từng trường dữ liệu

## 3.1.Post_date

In [54]:
# Loại các quan sát có thời điểm đăng quá 1 tháng để không bị out-of-date
car = car.loc[car["post_date"]>"2024-03-03",:]
car.shape

(42148, 14)

## 3.2.Tình trạng và số km đã đi

In [55]:
car.loc[car["mileage_v2"].isna(),["web","condition_ad"]].value_counts()

web      condition_ad
bonbanh  Mới             6245
         Đã sử dụng      2441
chotot   Mới             2105
         Đã sử dụng       258
Name: count, dtype: int64

In [56]:
# Gán các trường hợp Mới có mileage_v2=0 và drop các trường hợp trống thông tin số km với Đã sử dụng
car.loc[car["condition_ad"]=="Mới","mileage_v2"] = 0
car.dropna(subset="mileage_v2", axis=0, inplace=True)
car.shape

(39449, 14)

In [57]:
# Chuyển dạng dữ liệu
car["mileage_v2"] = car["mileage_v2"].astype(int)
car["mfdate"] = car["mfdate"].astype(int)

## 3.3.Hãng xe và dòng xe

In [58]:
# Kiểm tra giá trị trong trường
car[["carbrand","carmodel"]].sort_values(by=["carbrand","carmodel"]).groupby(["carbrand","carmodel"]).size().to_clipboard()

In [59]:
car.shape

(39449, 14)

In [60]:
# Thống nhất dòng xe và loại các dòng xe có dưới 10 quan sát (không đảm bảo thống kê)
car = car[~(car["carbrand"].isin(["Alfa Romeo","Aston Martin","Baic","Changan","Chery","Chrysler","Citroen",
                                  "Daihatsu","Dodge","Dongben","Dongfeng","Ferrari","GMC","Gaz","Geely","Genesis",
                                  "Haima","Hãng khác","Hino","HongQi","Hongqi","Infiniti","JRD","Lamborghini",
                                  "Lifan","Lincoln","Luxgen","Lynk & Co","Lynk&Co","Maserati","Maybach","McLaren",
                                  "Mekong","Morgan","Proton","Renault","SYM","Samsung","Smart","TQ Wuling","Teraco",
                                  "Thaco","Tobe","UAZ","Vinaxuki","Zotye"]))]

car = car[~((car["carbrand"]=="Acura") & (car["carmodel"]!="MDX"))]
car = car[~((car["carbrand"]=="Audi") & (car["carmodel"].isin(["A4","A5","A6","A8","Q2","Q3","Q5","Q7","Q8"])==False))]
car = car[~((car["carbrand"]=="BMW") & (car["carmodel"].isin(["3 Series","4 Series","5 Series","7 Series","X1","X3","X4","X5","X6","X7","Z4"])==False))]
car = car[~((car["carbrand"]=="Bentley") & (car["carmodel"]=="Azure"))]
car = car[~((car["carbrand"]=="Cadillac") & (car["carmodel"]!="Escalade"))]
car = car[~((car["carbrand"]=="Chevrolet") & (car["carmodel"].isin(["Camaro","Chevyvan","Dòng khác","Matiz","Nubira","Trax"])))]
car = car[~((car["carbrand"]=="Daewoo") & (car["carmodel"].isin(["Gentra","GentraX","Lacetti","Lanos","Magnus","Matiz","Nubira"])==False))]
car = car[~((car["carbrand"]=="Fiat") & (car["carmodel"]!="Siena"))]

car = car[~((car["carbrand"]=="Ford") & (car["carmodel"].isin(["Acononline","Dòng khác","F350","Flex","Imax"])))]
car["carmodel"].replace({"F 150":"F150"},inplace=True)

car = car[~((car["carbrand"]=="Honda") & (car["carmodel"].isin(["Dòng khác","Insight","Shuttle"])))]
car["carmodel"].replace({"BR V":"BR-V","CR V":"CRV","HRV":"HR-V"},inplace=True)

car = car[~((car["carbrand"]=="Hyundai") & (car["carmodel"].isin(["Azera","Click","Coupe","Dòng khác","Eon","Equus","Excel","Galloper","Sota","H 1","Khác","Libero",
                                                                  "Tiburon","Trajet","Tuscani","Universe","Universe Xpress Luxury","Veracruz","Verna","XG","Xcent"])))]
car["carmodel"].replace({"Santa Fe":"SantaFe","Grand i10":"i10"},inplace=True)

car = car[~((car["carbrand"]=="Isuzu") & (car["carmodel"].isin(["Dmax","Hi lander","MU-X","Mu X","QKR"])==False))]
car["carmodel"].replace({"Mu X":"MU-X"},inplace=True)

car = car[~((car["carbrand"]=="Jaguar") & (car["carmodel"].isin(["F Type","XE"])))]
car["carmodel"].replace({"F PACE":"F-Pace"},inplace=True)

car = car[~((car["carbrand"]=="Jeep") & (car["carmodel"].isin(["A2","Wrangler"])==False))]
car = car[~((car["carbrand"]=="Kia") & (car["carmodel"].isin(["Bongo","Cadenza","Dòng khác","Enterprise","Frontier","Khác","K3000S","Quoris","Ray","Sephia","Soul"])))]
car = car[~((car["carbrand"]=="LandRover") & (car["carmodel"].isin(["Freelander","Range Rover Vogue"])))]

car = car[~((car["carbrand"]=="Lexus") & (car["carmodel"].isin(["Dòng khác","RC"])))]
car["carmodel"].replace({"LM350":"LM","LX 600":"LX"},inplace=True)

car = car[~((car["carbrand"]=="MG") & (car["carmodel"].isin(["Dòng khác"])))]

car = car[~((car["carbrand"]=="Mazda") & (car["carmodel"].isin(["Atenza","Dòng khác","Premacy"])))]
car["carmodel"].replace({"BT 50":"BT50","CX 3":"CX3","CX 30":"CX-30","CX 5":"CX5","CX 8":"CX8","CX 9":"CX9"},inplace=True)

car = car[~((car["carbrand"]=="Mercedes Benz") & (car["carmodel"].isin(["190","AMG","AMG GT","B Class","CL Class","Dòng khác","CLK Class","CLS Class",
                                                                        "EQE","GT Coupe","M Class","MB","SL Class","SLC","SLK Class"])))]
car["carmodel"].replace({"A class":"A Class","C class":"C Class","CLA class":"CLA Class","E class":"E Class","G class":"G Class","GL":"GL Class",
                         "GLA class":"GLA Class","GLC":"GLC Class","GLS":"GLS Class","R class":"R Class","S class":"S Class","V class":"V Class"},inplace=True)

car = car[~((car["carbrand"]=="Mini") & (car["carmodel"].isin(["Cooper"])==False))]

car = car[~((car["carbrand"]=="Mitsubishi") & (car["carmodel"].isin(["Attrage","Grandis","Jolie","Lancer","Mirage","Outlander","Outlander Sport","Pajero",
                                                                     "Pajero Sport","Triton","XFC","Xforce","Xpander","Xpander Cross"])==False))]
car["carmodel"].replace({"Outlander Sport":"Outlander"},inplace=True)

car = car[~((car["carbrand"]=="Nissan") & (car["carmodel"].isin(["Almera","Grand livina","Kicks","Livina","Navara","Sunny","Teana","Terra",
                                                                "X Terra","X trail"])==False))]
car["carmodel"].replace({"Grand livina":"Livina","X Terra":"Terra","X trail":"X Trail"},inplace=True)

car = car[~((car["carbrand"]=="Peugeot") & (car["carmodel"].isin(["107","405","508","RCZ"])))]
car = car[~((car["carbrand"]=="Porsche") & (car["carmodel"].isin(["Boxster","Cayman","Dòng khác","Taycan"])))]
car = car[~((car["carbrand"]=="Rolls Royce") & (car["carmodel"].isin(["Dawn"])))]
car = car[~((car["carbrand"]=="Ssangyong") & (car["carmodel"].isin(["Stavic"])==False))]
car = car[~((car["carbrand"]=="Subaru") & (car["carmodel"].isin(["WRX"])))]

car = car[~((car["carbrand"]=="Suzuki") & (car["carmodel"].isin(["Carry","Dòng khác","Jimny","Super Carry Truck","Wagon R+"])))]
car["carmodel"].replace({"Grand vitara":"Vitara","XL 7":"XL7"},inplace=True)

car = car[~((car["carbrand"]=="Toyota") & (car["carmodel"].isin(["1116","4 Runner","Aurion","Aygo","Celica","Cressida","Dòng khác","IQ",
                                                                 "Khác","Mark II","Matrix","Previa","Wish","Yaris Verso"])))]
car["carmodel"].replace({"Corolla altis":"Corolla Altis","XL 7":"XL7"},inplace=True)

car["carbrand"].replace({"Vinfast":"VinFast"},inplace=True)
car = car[~((car["carbrand"]=="VinFast") & (car["carmodel"].isin(["Dòng khác","VF3","1189"])))]
car["carmodel"].replace({"Lux A2.0":"Lux A 2.0","Lux SA2.0":"Lux SA 2.0","VF5 Plus":"VF5","VFe34":"VF e34"},inplace=True)

car = car[~((car["carbrand"]=="Volkswagen") & (car["carmodel"].isin(["Passat","Polo","T-Cross","Teramont","Teramont X","Tiguan","Touareg","Viloran","Virtus"])==False))]
car = car[~((car["carbrand"]=="Volvo") & (car["carmodel"].isin(["S90","XC60","XC90"])==False))]

In [61]:
car[["carbrand","carmodel"]].sort_values(by=["carbrand","carmodel"]).groupby(["carbrand","carmodel"]).size().to_clipboard()
car.shape

(38339, 14)

## 3.3.Hộp số

In [62]:
# Kiểm tra giá trị trong trường
car[["web","gearbox"]].value_counts()

web      gearbox    
bonbanh  Tự động        19785
chotot   Tự động        12386
         Số sàn          3967
bonbanh  Số sàn          2151
chotot   Bán tự động       50
Name: count, dtype: int64

In [63]:
# Chuyển "Bán tự động" thành "Tự động"
car["gearbox"].replace({"Bán tự động":"Tự động"}, inplace=True)
car[["web","gearbox"]].value_counts()

web      gearbox
bonbanh  Tự động    19785
chotot   Tự động    12436
         Số sàn      3967
bonbanh  Số sàn      2151
Name: count, dtype: int64

## 3.4.Nhiên liệu

In [64]:
# Kiểm tra giá trị trong trường
car[["web","fuel"]].value_counts()

web      fuel          
bonbanh  Xăng              17653
chotot   Xăng              13474
bonbanh  Dầu                3542
chotot   Dầu                2631
bonbanh  Động cơ Hybrid      381
         Điện                359
chotot   Điện                154
         Động cơ Hybrid      144
Name: count, dtype: int64

In [65]:
# Kiểm tra trường null
car.loc[car["fuel"].isna(),:]

# Kiểm tra thì động cơ dầu, gán là dầu
car.loc[car["fuel"].isna(),"fuel"] = "Dầu"

## 3.5.Nơi sản xuất

In [66]:
# Kiểm tra giá trị trong trường
car[["web","carorigin"]].value_counts()

web      carorigin         
bonbanh  Lắp ráp trong nước    12189
         Nhập khẩu              9747
chotot   Việt Nam               7754
         Thái Lan               2155
         Nước khác              1311
         Hàn Quốc                922
         Nhật Bản                876
         Đức                     472
         Mỹ                      387
         Ấn Độ                   170
         Trung Quốc               31
         Đài Loan                 28
Name: count, dtype: int64

In [67]:
# Gán các trường hợp trong bộ chotot để thống nhất với bộ bonbanh
car.loc[(car["web"]=="chotot") & (car["carorigin"].isna()==False),"carorigin"] = car.loc[(car["web"]=="chotot") & (car["carorigin"].isna()==False),"carorigin"].apply(lambda x: "Lắp ráp trong nước" if x == "Việt Nam" else "Nhập khẩu")
car[["web","carorigin"]].value_counts()

web      carorigin         
bonbanh  Lắp ráp trong nước    12189
         Nhập khẩu              9747
chotot   Lắp ráp trong nước     7754
         Nhập khẩu              6352
Name: count, dtype: int64

In [68]:
# Kiểm tra trường hợp NaN
car.loc[car["carorigin"].isna(),"web"].value_counts()

web
chotot    2297
Name: count, dtype: int64

In [69]:
# Thống kê carorigin có quan sát nhiều nhất theo từng model
full_origin = car.loc[:,["carmodel","carbrand","carorigin"]].value_counts().reset_index().rename(columns={"count":"count_full"})
full_origin_max = full_origin.groupby(by=["carbrand","carmodel"])["count_full"].max().reset_index(name="max_count")
full_origin = full_origin.merge(full_origin_max)
full_origin = full_origin[full_origin["count_full"]==full_origin["max_count"]]
print(full_origin.shape)

# Thống kê và merge các trường hợp NaN
check_origin = car.loc[car["carorigin"].isna(),["carmodel","carbrand"]].value_counts().reset_index()
print(check_origin.shape)

check_origin = check_origin.merge(full_origin)
print(check_origin.shape)
# --> Tồn tại một xe có 2 giá trị max
check_origin[["carbrand","carmodel"]].value_counts().sort_values(ascending=False)

(282, 5)
(208, 3)
(209, 6)


carbrand    carmodel
Mazda       626         2
Audi        A4          1
BMW         X5          1
Audi        Q2          1
            Q3          1
                       ..
VinFast     VF9         1
Volkswagen  Passat      1
            Polo        1
Volvo       S90         1
            XC90        1
Name: count, Length: 208, dtype: int64

In [70]:
# Vá các trường hợp Nan
car.loc[(car["carorigin"].isna()) & (car["carmodel"]!="626"),"carorigin"] = list(car.loc[(car["carorigin"].isna()) & (car["carmodel"]!="626"),["carbrand","carmodel"]].merge(full_origin)["carorigin"])
car.loc[car["carorigin"].isna(),"web"].value_counts()

web
chotot    2
Name: count, dtype: int64

In [71]:
# Drop các bản ghi còn NA
car = car.dropna(subset=["carorigin"])
car.shape

(38337, 14)

## 3.6.Kiểu dáng, số chỗ ngồi

In [72]:
# Kiểm tra nan trong bộ dữ liệu
car.isna().sum().sort_values(ascending=False)

carseats        16043
cartype         13206
url                 0
post_date           0
Price               0
carbrand            0
carmodel            0
mfdate              0
mileage_v2          0
condition_ad        0
gearbox             0
fuel                0
carorigin           0
web                 0
dtype: int64

In [73]:
# carseats và cartype null nhiều, nếu vá bằng carmodel thì chỉ vá được các trường hợp carmodel tương ứng 1 cartype.
# Tuy nhiên như vậy thì vẫn phụ thuộc vào carmodel
# --> Drop hai trường này
car.drop(["carseats","cartype"], axis=1, inplace=True)

In [74]:
# Export data to pickle
with open('car_cleaned.pkl','wb') as f:
    pickle.dump(car,f)